In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18

# Define transforms to apply to the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 (input size for ResNet)
    transforms.ToTensor(),         # Convert images to tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize images
])

# Load dataset
dataset = ImageFolder(root='plantvillage', transform=transform)

# Split dataset into training and testing sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
classes = dataset.classes
print("Classes:", classes)



Classes: ['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'cotton_bacterial_blight', 'cotton_curl_virus', 'cotton_fussarium_wilt', 'cotton_healthy']


In [2]:
resnet_model = resnet18(pretrained=True)

# Freeze parameters of the pre-trained model
for param in resnet_model.parameters():
    param.requires_grad = False

# Modify the last fully connected layer to match the number of classes
num_ftrs = resnet_model.fc.in_features
resnet_model.fc = nn.Linear(num_ftrs, len(dataset.classes))

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet_model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet_model.to(device)

for epoch in range(num_epochs):
    resnet_model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = resnet_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
    
    epoch_loss = running_loss / len(train_dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")


Epoch [1/10], Loss: 0.8743
Epoch [2/10], Loss: 0.3376
Epoch [3/10], Loss: 0.2225
Epoch [4/10], Loss: 0.1739
Epoch [5/10], Loss: 0.1561
Epoch [6/10], Loss: 0.1322
Epoch [7/10], Loss: 0.1127
Epoch [8/10], Loss: 0.0958
Epoch [9/10], Loss: 0.0905
Epoch [10/10], Loss: 0.0794


In [3]:
resnet_model.eval()
all_preds = []
all_labels = []
for images, labels in test_loader:
    images, labels = images.to(device), labels.to(device)
    outputs = resnet_model(images)
    _, preds = torch.max(outputs, 1)
    all_preds.extend(preds.cpu().numpy())
    all_labels.extend(labels.cpu().numpy())

In [4]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(all_labels, all_preds)
print(f"Accuracy on test set: {accuracy:.4f}")

Accuracy on test set: 0.9793


In [5]:
torch.save(resnet_model.state_dict(), "model.pth")